<a href="https://colab.research.google.com/github/yussif-issah/css54FinalProject/blob/main/visontransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers[torch]

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers datasets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import torchvision
from torchvision import transforms
from datasets import load_dataset
from transformers import ViTImageProcessor,ViTForImageClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score,confusion_matrix ,ConfusionMatrixDisplay
import matplotlib.pyplot as plt

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [4]:
train_dataset = load_dataset("imagefolder", data_dir="datamangrove/train",split="train")
validation_dataset = load_dataset("imagefolder", data_dir="datamangrove/val",split="train")
test_dataset = load_dataset("imagefolder", data_dir="datamangrove/test",split="train")

Resolving data files:   0%|          | 0/208 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Resolving data files:   0%|          | 0/61 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Resolving data files:   0%|          | 0/31 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
itos = dict((k,v) for k,v in enumerate(train_dataset.features['label'].names))
stoi = dict((v,k) for k,v in enumerate(train_dataset.features['label'].names))
itos

{0: 'Avicennia Alba', 1: 'Rhizophora Apiculata', 2: 'Sonneratia Alba'}

In [6]:
model_name = "google/vit-base-patch16-224"
processor = ViTImageProcessor.from_pretrained(model_name)

mu, sigma = processor.image_mean, processor.image_std #get default mu,sigma
size = processor.size

/home/yussif/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [7]:
norm = transforms.Normalize(mean=mu, std=sigma) #normalize image pixels range to [-1,1]

# resize 3x32x32 to 3x224x224 -> convert to Pytorch tensor -> normalize
_transf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    norm
])

# apply transforms to PIL Image and store it to 'pixels' key
def transf(arg):
    arg['pixels'] = [_transf(image.convert('RGB')) for image in arg['image']]
    return arg

In [8]:
model = ViTForImageClassification.from_pretrained(model_name, num_labels=3, ignore_mismatched_sizes=True, id2label=itos, label2id=stoi)
print(model.classifier)

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([3]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Linear(in_features=768, out_features=3, bias=True)


In [9]:
train_dataset.set_transform(transf)
validation_dataset.set_transform(transf)
test_dataset.set_transform(transf)


In [10]:
args = TrainingArguments(
    f"test ",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir='logs',
    remove_unused_columns=False,
)

NotImplementedError: Using RTX 4000 series doesn't support faster communication broadband via P2P or IB. Please set `NCCL_P2P_DISABLE="1"` and `NCCL_IB_DISABLE="1" or use `accelerate launch` which will do this automatically.

In [ ]:
def collate_fn(examples):
    pixels = torch.stack([example["pixels"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixels, "labels": labels}

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return dict(accuracy=accuracy_score(predictions, labels))

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.182124,1.000000
2,No log,0.040148,1.000000
3,No log,0.025199,1.000000


TrainOutput(global_step=63, training_loss=0.22020106845431858, metrics={'train_runtime': 167.4476, 'train_samples_per_second': 3.727, 'train_steps_per_second': 0.376, 'total_flos': 4.835543491008922e+16, 'train_loss': 0.22020106845431858, 'epoch': 3.0})

In [ ]:
outputs = trainer.predict(test_dataset)
outputs.metrics

{'test_loss': 0.18618705868721008,
 'test_accuracy': 1.0,
 'test_runtime': 16.4897,
 'test_samples_per_second': 1.88,
 'test_steps_per_second': 0.485}